In [ ]:
import numpy as np  
import cv2  
import dlib  
from scipy.spatial import distance as dist  
   
PREDICTOR_PATH = "../Data/Misc/shape_predictor_68_face_landmarks.dat"  
   
RIGHT_EYE_POINTS = list(range(36, 42)) 
LEFT_EYE_POINTS = list(range(42, 48))  

   
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 2  
frame_c=0   
COUNTER_LEFT = 0  
TOTAL_LEFT = 0  
   
COUNTER_RIGHT = 0  
TOTAL_RIGHT = 0  
   
def eye_aspect_ratio(eye):  
    """
    Based on the work by Soukupová and Čech in their 2016 paper
    http://vision.fe.uni-lj.si/cvww2016/proceedings/papers/05.pdf
    """
    A = dist.euclidean(eye[1], eye[5])  
    B = dist.euclidean(eye[2], eye[4])   
    C = dist.euclidean(eye[0], eye[3])  
    ear = (A + B) / (2.0 * C)
    return ear  
   
detector = dlib.get_frontal_face_detector()  
predictor = dlib.shape_predictor(PREDICTOR_PATH)  
  
video_capture = cv2.VideoCapture(0)  
   
while True:
    global frame_c
    frame_c +=1
    ret, frame = video_capture.read()  
   
    if ret:  
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  
   
        rects = detector(gray, 0)  

        for rect in rects:

            landmarks = np.matrix([[p.x, p.y] for p in predictor(frame, rect).parts()])  

            left_eye = landmarks[LEFT_EYE_POINTS]  
            right_eye = landmarks[RIGHT_EYE_POINTS]  

            left_eye_hull = cv2.convexHull(left_eye)  
            right_eye_hull = cv2.convexHull(right_eye)
            
            cv2.drawContours(frame, [left_eye_hull], -1, (0, 255, 0), 1)  
            cv2.drawContours(frame, [right_eye_hull], -1, (0, 255, 0), 1)  

            ear_left = eye_aspect_ratio(left_eye)  
            ear_right = eye_aspect_ratio(right_eye)  

            cv2.putText(frame, "E.A.R. Left : {:.2f}".format(ear_left), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  
            cv2.putText(frame, "E.A.R. Right: {:.2f}".format(ear_right), (300, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  

            if ear_left < EYE_AR_THRESH:  
                COUNTER_LEFT += 1  
            else:  
                if COUNTER_LEFT >= EYE_AR_CONSEC_FRAMES:  
                    TOTAL_LEFT += 1  
                COUNTER_LEFT = 0  

            if ear_right < EYE_AR_THRESH:  
                COUNTER_RIGHT += 1  
            else:  
                if COUNTER_RIGHT >= EYE_AR_CONSEC_FRAMES:  
                    TOTAL_RIGHT += 1
                COUNTER_RIGHT = 0  

            cv2.putText(frame, "Wink Left : {}".format(TOTAL_LEFT), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)   
            cv2.putText(frame, "Wink Right: {}".format(TOTAL_RIGHT), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  

            cv2.imshow("Faces found", frame)    

    if cv2.waitKey(1) & 0xFF == ord('q'):  
        break

video_capture.release()
cv2.destroyAllWindows()  